##Distributions:

###Claim Severity Distribution:

Gamma distribution with $\mu = 4$, $ \sigma^2 = 6$.

$\mu = \frac{\alpha}{\lambda} = 4$.

$\sigma^2 = \frac{\alpha}{\lambda^2} = 6$.

$\longrightarrow \alpha = \frac{4}{9} , \lambda = \frac{1}{9}$.

###Claim Frequency Distribution:

Poisson distribution with ${\mu} = {\lambda} = 5$.



##Default Parameters:

In [ ]:
#gamma
alpha = 4/9
beta = 1/9   #(named beta because lambda is a keyword in python)

#poisson
mu = 5

##These sliders can be used to set the random variable parameters: 

0 means default parameters (alpha = 4/9, lambda = 1/9 for the gamma and 5 for the poisson).

Please run this code cell (or run all) after modifying the parameters.

In [ ]:
gamma_alpha_slider = 0 #@param {type:"slider", min:0, max:100, step:0.1}
gamma_lambda_slider = 0 #@param {type:"slider", min:0, max:100, step:0.1}
poisson_lambda_slider = 0 #@param {type:"slider", min:0, max:100, step:0.1}

if gamma_alpha_slider != 0:
  alpha = gamma_alpha_slider

if gamma_lambda_slider != 0:
  beta = gamma_lambda_slider

if poisson_lambda_slider != 0:
  mu = poisson_lambda_slider

In [ ]:
#parameters for Panjer's Formula
a = 0
b = beta

##Import Statements:

In [ ]:
import math
from scipy.stats import gamma
import numpy as np
import pandas as pd

##Discretized Gamma:

This function takes a natural number n, and returns the probability that a gamma(alpha, lambda) RV is in the range (n - .5, n + .5).

In [ ]:
def discretized_gamma(x):
  # the scale parameter in this library is the reciprocal of beta or lambda. Source: https://stackoverflow.com/a/42151088
  gamma_rv = gamma(a=alpha, scale=(1/beta))
  return gamma_rv.cdf(x + .5) - gamma_rv.cdf(x - .5)

##Panjer's Formula:

$P_N (z)=e^{(λ(z-1))} =e^{(5(z-1))}$

$g_x=\frac{1}{(1-af_0 )} ∑_{(y=1)}^x(a+\frac{b}{x} y)  f_y g_{(x-y)}$

$g_0 = P_N (f_0 ) = e^{(λ(f_0-1))} $

$poisson => a = 0, b = {\mu} = {\lambda} = 5$

$∑_{(y=1)}^x\frac{5}{x} y f_y g_{(x-y)}$

$g_0 = P_N (f_0 ) = e^{(5(f_0-1))}$





##Panjer's Recursive Formula First Attempt:

This version cannot efficiently calculate more than the first few probabilities. ( $O(n!)$ running time)

In [ ]:
def get_compound_probability(x):

  #base case: total = 0.
  if x == 0:
    return get_probability_that_total_is_zero()
  
  #recursive case:
  sum = 0
  for y in range(1, x + 1):
    sum += ((b/x) * y) * discretized_gamma(y) * get_compound_probability(x - y)
  return sum


#This "helper function" handles the base case for both versions:
def get_probability_that_total_is_zero():

  def evaluate_poisson_pgf(z):
    return math.exp(beta * (z - 1))

  f_0 = discretized_gamma(0)
  result = evaluate_poisson_pgf(f_0)
  return result

##Panjer's Recursive Formula Memoized Version:

This version uses a dictionary to store previously computed probabilities so they are not recomputed.

In [ ]:
probabilities = {}

def memoized_compound_probability(x):
  if x in probabilities:
    return probabilities[x] 
  elif x == 0:
    p = get_probability_that_total_is_zero()
    probabilities[x] = p
    return p
  else:
    sum = 0
    for y in range(1, x + 1):
      sum += ((b/x) * y) * discretized_gamma(y) * memoized_compound_probability(x - y)
    probabilities[x] = sum
    return sum

##Test

In [ ]:
#Tests that the sum of probabilities of all outcomes approachs one.
sum = 0
for i in range(100):
  sum += memoized_compound_probability(i)
np.testing.assert_almost_equal(sum, 1, decimal=5)

##Calculating the first 40 probabilities:

In [ ]:
file_path = 'panjers_formula_results.csv'

probabilities = {}

for i in range(40):
  probability = memoized_compound_probability(i)
  probabilities[i] = probability
  print("probability of {} = {}".format(i, probability))


probability of 0 = 0.9259113493469866
probability of 1 = 0.018217407704284247
probability of 2 = 0.010903077655781558
probability of 3 = 0.007826511732431029
probability of 4 = 0.006008347099645286
probability of 5 = 0.004780447741567624
probability of 6 = 0.003889255286751366
probability of 7 = 0.003212845532416686
probability of 8 = 0.002683694357169751
probability of 9 = 0.0022606848265277753
probability of 10 = 0.0019170159359963546
probability of 11 = 0.0016343032372874813
probability of 12 = 0.0013994280199100255
probability of 13 = 0.0012027321203199718
probability of 14 = 0.0010369238835556808
probability of 15 = 0.000896383854112924
probability of 16 = 0.0007767068703461063
probability of 17 = 0.0006743900761418831
probability of 18 = 0.0005866143437732323
probability of 19 = 0.0005110874086900919
probability of 20 = 0.0004459289095695074
probability of 21 = 0.00038958457957122077
probability of 22 = 0.0003407611546907839
probability of 23 = 0.0002983762881422651
probability o